In [1]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time
import csv

# File Name

In [2]:
tr_file = "5Act_Var_Tr1_JH_Wifi_raw_complete"
te_file = "5Act_Var_Te1_JH_Wifi_raw_complete"

# variable 

In [14]:
ap_address = []
ap_signal = []
location = []
ap_lev = []
ap_lev_times = []
selected_ap_index = []

# Function ==================================

file은 한번 만 읽어서 각 array에 저장한 후에 처리하도록 함(같은 파일을 다시 읽지 않음)

In [4]:
def read_file(filename,ap_address, ap_signal, location):
    filename =  "data/testfornn/"+filename+".csv"
    with open(filename, 'rb') as csvfile:
        reader = csv.reader (csvfile, delimiter = ',')
        for row in reader:
            ap_address.append(row[2])
            ap_signal.append(row[3])
            location.append(row[4])

총 ap_address 들을 구해줌

In [26]:
def get_ap_lev(ap_address,ap_lev):
    for i in range(0,len(ap_address)):
        if ap_address[i] not in ap_lev:
            ap_lev.append(ap_address[i])
    return ap_lev

해당 ap가 총 몇 회 불렸는지 확인하는 함수 해당 함수는 get_ap_lev 함수 실행 이후에 돌려야함

In [6]:
def get_ap_levtimes(ap_address,ap_lev,ap_lev_times):
    ap_lev_times = np.zeros(len(ap_lev),dtype=np.int)
    for i in range(0, len(ap_address)):
        index = ap_lev.index(ap_address[i])
        ap_lev_times[index] += 1
    return ap_lev_times

ap_signal의 등장횟수가 많은 것을 기준으로 NN에 이용할 AP를 선택하는 함수

In [20]:
def select_ap_address(data_max,ap_lev_times, selected_ap_index):
    threshold = data_max # data_max *0.05
    for i in range(0,len(ap_lev_times)):
        if ap_lev_times[i] >= threshold:
            selected_ap_index.append(i)
    return threshold

data의 수가 가장 많은 것을 기준으로, 해당 data의 등장후 다음 등장 이전까지를 하나의 epoch으로 지정

위의 흐름대로 진행할 때 데이터 중에서 다른 location 데이터와 함께 들어와 있는 경우, 
무조건 행에 따라서 data를 나열하면 input(ap_signal)과 output(location)에 혼선이 있을 우려가 있음

[0 1 2 0 1 2 0 1] 로 등장하였다고 가정하고, 해당 하는 location 이<br>[5 5 5 5 5 5 5 4] 로 등장하였을 경우, 

마지막 두 개의 ap_address_index에서 0 1 이 다른 location인 경우가 발생되는 것을 방지하고자 함

우선 location 별로 partition을 생성한 다음, 각 파티션 별로 ap selected array를 구현한 다음 다시 합쳐지는 방식을 이용함


1. (선택된 ap index) x (data_max) 행렬
2. (location) x (data_max) 행렬

In [ ]:
def location_partition(ap_address, ap_signal,ap_lev, selected_ap_index, location):
    epoch_benchmark = ap_lev[selected_ap_index[0]]
    for i in range(0,len(ap_address)):
        

location을 softmax함수에 이용할 수 있도록 변환 <br>
1 => [0,1,0,0]<br>
3 => [0,0,0,1] 식으로 변환하는 함수

In [21]:
def make_location_to_binary_array(data_max,location, bi_location):
    location_num = location[np.argmax(location)]
    bi_location = np.zeros((data_max,location_num))
    for i in range(0,data_max):
        bi_location[i,location(i)] = 1
    return location_num

# Function End================================

ap의 level과 총 level갯수

In [22]:
read_file(tr_file,ap_address, ap_signal, location)
print get_ap_lev(ap_address, ap_lev)
print len(ap_lev)

['00:26:66:f4:d0:58', '08:5d:dd:43:8e:ef', '00:07:89:3f:fc:28', '90:9f:33:19:82:2e', '08:5d:dd:11:39:20', '08:5d:dd:0a:30:6a', '90:9f:33:8f:f5:08', '90:9f:33:40:4a:02', '00:30:0d:15:7c:30', 'bc:96:80:98:67:01', '04:8d:38:3d:a4:74', 'bc:96:80:98:67:04', '00:30:0d:a9:f7:1e', '00:07:89:17:22:1f', '90:9f:33:3d:3d:4a', '10:f9:6f:f4:4c:69', '64:e5:99:bb:01:12', '00:08:9f:92:40:d8', '00:08:52:21:1b:4c', '10:f9:6f:f4:4c:6a', '00:40:5a:c1:28:62', 'f0:b4:29:1a:62:6d', '9c:d6:43:af:41:d4', '00:40:5a:c1:28:61', '00:08:9f:40:0b:6f', '64:e5:99:e1:a5:1a', '00:26:66:f4:06:20', '18:53:e0:01:49:bd', 'c0:a0:bb:59:a6:b8', '00:07:89:17:f6:c2', '00:07:89:46:79:45', '90:9f:33:92:73:54', '90:9f:33:6c:cb:9e', '00:08:52:36:2d:43', '00:17:c3:9f:b6:1d', 'ac:a2:13:54:06:98']
36


In [23]:
ap_lev_times = get_ap_levtimes(ap_address,ap_lev,ap_lev_times)
print ap_lev_times
print len(ap_lev_times)

[289 289 289 256 217 215 191  46 145  41  27  32 154 155  46  91  80  73
  81  57  50  73  49  89  59   2  19   8  12   2   8  51  19   2   1  15]
36


In [24]:
data_max = ap_lev_times[np.argmax(ap_lev_times)]

In [25]:
print select_ap_address(data_max,ap_lev_times, selected_ap_index)
print selected_ap_index

289
[0, 1, 2]
